In [1]:
from pulp import *
import csv
import numpy as np
import pandas as pd

In [2]:
in_file = "D:/Datasets/DKSalariesOdds.csv"
reader = csv.DictReader(open(in_file, 'r'))
player_dict = []
for i , row in enumerate(reader):
    t = {k: v for k, v in row.items()}
    t['id'] = i
    player_dict.append(t)
    
# Create an empty LP
IP = LpProblem("Golf Optimization", LpMinimize)

# Create a list of the teams and player IDs
teams = [i for i in range(10)]
players = [player['id'] for player in player_dict]

# Create binary decision variables for which player ends up on which team
possible_assignments = LpVariable.dicts("x", (teams, players), 0, 1, LpBinary)

# Add in the objective function
IP += sum([float(player['Odds']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])
#IP += 

for team in teams:
    IP += lpSum([possible_assignments[team][player['id']] for player in player_dict]) == 6, '6 players per team '+str(team)
    IP += sum([float(player['Salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
    #IP += sum([float(player['odds']) < 15 for player in player_dict]) > 0, '1 favorite per team'
    #IP += lpSum([possible_assignments[t][p['id']] for p in d if p['Group'] == 'A']) == 1, '1 A per team '+str(t)
    #IP += lpSum([possible_assignments[t][p['id']] for p in d if p['Group'] == 'B']) == 2, '2 B per team '+str(t)
    #IP += lpSum([possible_assignments[t][p['id']] for p in d if p['Group'] == 'C']) == 3, '3 C per team '+str(t)
    #IP += lpSum([possible_assignments[t][p['id']] for p in d if p['Group'] == 'D']) == 2, '2 D per team '+str(t)
    
    
    
    
# Write out the LP formulation
IP.writeLP('simple optimization.lp')

In [3]:
# Solve the LP
IP.solve()

# Print out the LP solution status
print("Status:", LpStatus[IP.status])

Status: Optimal


In [4]:
# Print out some results
print("The players chosen for team 1 are:")

opt_salary = []
opt_odds = []
for player in player_dict:
    if possible_assignments[1][player['id']].value() == 1.0:
        print(player['Name']+ '- odds: ' + player['Odds'] + ' Salary: ' + player['Salary'])
        opt_odds.append(float(player['Odds']))
        opt_salary.append(int(player['Salary']))

print('Total Salary - %i, Avg odds to win - %i' % (sum(opt_salary), np.mean(opt_odds)))
        

The players chosen for team 1 are:
Hideki Matsuyama- odds: 40.23 Salary: 9100
Louis Oosthuizen- odds: 30.78 Salary: 8800
Brandt Snedeker- odds: 46.78 Salary: 8500
Patrick Reed- odds: 47.48 Salary: 8300
Paul Casey- odds: 73.2 Salary: 7700
Zach Johnson- odds: 54.43 Salary: 7600
Total Salary - 50000, Avg odds to win - 48


In [5]:
#Cat food simplified Model
#Create the problem variable to contain the problem data
prob = LpProblem('The Whiskas Problem', LpMinimize)

#Create 2 variables Beef and Chicken with lower limit of 0
x1 = LpVariable('Chick Percent', 0, None, LpInteger)
x2 = LpVariable('Beef Percent', 0)

#Create the objective of the optimization
prob += 0.013 * x1 + 0.008 * x2, "Total Cost of Ingredients per can"

#Create 5 constraints
prob += x1 + x2 == 100, 'Percentage Sum'
prob += 0.100 * x1 + 0.200 * x2 >= 8.0, 'Protein Requirement'
prob += 0.080 * x1 + 0.100 * x2 >= 6.0, 'Fat Requirement'
prob += 0.001 * x1 + 0.005 * x2 <= 2.0, 'Fiber Requirement'
prob += 0.002 * x1 + 0.005 * x2 <= 0.4, 'Salt Rquirement'

In [6]:
# Solve problem
prob.solve()
print('Status: ', LpStatus[prob.status])

Status:  Optimal


In [7]:
for v in prob.variables():
    print(v.name, ' = ', v.varValue)

Beef_Percent  =  66.0
Chick_Percent  =  34.0


In [8]:
# The optimized objective function value is printed to the screen
print('Total Cost of Ingredients per can = ', value(prob.objective))

Total Cost of Ingredients per can =  0.97


In [9]:
#Cat food full version
# Create a list of ingredients
Ingredients = ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat', 'Gel']

# Dict of the cost of each ingredient
costs = {'Chicken': 0.013,
         'Beef': 0.008,
         'Mutton': 0.010,
         'Rice': 0.002,
         'Wheat': 0.005,
         'Gel': 0.001}

# Dict of protein percent
proteinPercent = {'Chicken': 0.100,
                  'Beef': 0.200,
                  'Mutton': 0.150,
                  'Rice': 0.000,
                  'Wheat': 0.040,
                  'Gel': 0.000}

# A dictionary of the fat percent
fatPercent = {'Chicken': 0.080,
              'Beef': 0.100,
              'Mutton': 0.110,
              'Rice': 0.010,
              'Wheat': 0.010,
              'Gel': 0.000}

# Dict of fiber
fiberPercent = {'Chicken': 0.001,
                'Beef': 0.005,
                'Mutton': 0.003,
                'Rice': 0.100,
                'Wheat': 0.150,
                'Gel': 0.000}

# Dict of salt percent
saltPercent = {'Chicken': 0.002,
               'Beef': 0.005,
               'Mutton': 0.007,
               'Rice': 0.002,
               'Wheat': 0.008,
               'Gel': 0.000}

In [10]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem('The Whiskas Problem', LpMinimize)

# Dict called ingredient vars to contain reference variables
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)

# The objective funct is added to prob first
prob += lpSum([costs[i] * ingredient_vars[i] for i in Ingredients]), 'Total Cost of Ingredients per can'

# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fiberPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"

In [11]:
# Solve problem
prob.solve()
print('Status: ', LpStatus[prob.status])

Status:  Optimal


In [12]:
for v in prob.variables():
    print(v.name, ' = ', v.varValue)

Ingr_Beef  =  60.0
Ingr_Chicken  =  0.0
Ingr_Gel  =  40.0
Ingr_Mutton  =  0.0
Ingr_Rice  =  0.0
Ingr_Wheat  =  0.0


In [13]:
# The optimized objective function value is printed to the screen
print('Total Cost of Ingredients per can = ', value(prob.objective))

Total Cost of Ingredients per can =  0.52


In [2]:
in_file = "D:/Datasets/DKSalariesOdds.csv"
reader = csv.DictReader(open(in_file, 'r'))
player_dict = []
for i , row in enumerate(reader):
    team = {k: v for k, v in row.items()}
    team['id'] = i
    player_dict.append(team)
    

In [3]:
teams = [i for i in range(10)]
players = [player['id'] for player in player_dict]


golf = LpProblem('Masters', LpMinimize)

possible_assignments = LpVariable.dicts("possible", (teams, players), 0, 1, LpBinary)

#objective
golf += sum([float(player['Odds']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])

for team in teams:
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict]) == 6, '6 players per team '+str(team)
    golf += sum([float(player['Salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if float(player['Odds']) < 8]) >= 1
    #golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if float(player['Odds']) < 20]) >= 2
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['Name'] == 'Dustin Johnson']) == 1
    #golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['Name'] == 'Zach Johnson']) == 0

In [4]:
golf.writeLP('masters_optimization.lp')
golf.solve()
print('Status: ', LpStatus[golf.status])


Status:  Optimal


In [5]:
# Print out some results
print("The players chosen for team 1 are:")

opt_salary = []
opt_odds = []
for player in player_dict:
    if possible_assignments[1][player['id']].value() == 1.0:
        print(player['Name']+ '- odds: ' + player['Odds'] + ' Salary: ' + player['Salary'])
        opt_odds.append(float(player['Odds']))
        opt_salary.append(int(player['Salary']))

print('Total Salary - %i, Avg odds to win - %i' % (sum(opt_salary), np.mean(opt_odds)))
        

The players chosen for team 1 are:
Jason Day- odds: 7.85 Salary: 11600
Dustin Johnson- odds: 18.28 Salary: 10200
Zach Johnson- odds: 54.43 Salary: 7600
Harris English- odds: 123.83 Salary: 7100
Keegan Bradley- odds: 137.42 Salary: 6900
Charley Hoffman- odds: 113.95 Salary: 6400
Total Salary - 49800, Avg odds to win - 75
